In [ ]:
!apt-get install sox ffmpeg libcairo2 libcairo2-dev
!pip install igraph leidenalg pycairo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcairo2 is already the newest version (1.16.0-4ubuntu1).
libcairo2-dev is already the newest version (1.16.0-4ubuntu1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2build0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import pickle
import json
import time
import re
import matplotlib.pyplot as plt
import igraph as ig
import leidenalg as la
import networkx as nx

In [ ]:
data = pd.read_csv('20230614T163906___zepto__blr_whitefield.binning.csv', sep='|')

In [ ]:
order_df = pd.read_csv('test_weekly_all_region.order.csv',sep='|')

In [ ]:
data.head()

,indent_date,client_id,store_id,region_id,zone_id,peak_id,sku_id,crate_type,subsection_type,subsection_id,order_type,qty_in_subsection,orientation(along_crate_lbh),expected_order_composition,raw_sku_order_distribution,raw_expected_sku_order_distribution,rationalised_order_qty_raw_order_qty_map,rationalised_sku_order_distribution,rationalised_expected_sku_order_distribution,subsection_peak_time_order_affinity
0,20221128,zepto,blr_whitefield,1,1,2,88449a9b_7f66_49a6_b24f_9d282ad4a763,LARGE_CRATE,QUARTER_SUBSECTION,1,BULK,284.0,"(143, 78, 36)",{4: 71.07718894009219},{4: 15.571428571428573},"{4: 28.542627728110602, 6: 23.557604686635948,...","{4: [4, 6, 8, 12]}",na,{4: 71.07718894009219},0
1,20221128,zepto,blr_whitefield,1,1,2,ec75d477_2f83_4015_92bd_57dc0f777076,LARGE_CRATE,OCTET_SUBSECTION,3,BULK,88.0,"(120, 40, 5)",{3: 29.333333333333332},{3: 6.666666666666667},"{4: 3.1428581428571434, 2: 23.25714385714286, ...","{3: [4, 2, 3, 6, 1]}",na,{3: 29.333333333333332},0
2,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,5,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
3,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,6,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
4,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,7,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0


In [ ]:
d = {2:[1,2], 3:[3,4,5], 6:[6,7,8]}
a = list(d.values())
a = [element for sublist in list(d.values()) for element in sublist]

print(a)
print(d.get(6))

[1, 2, 3, 4, 5, 6, 7, 8]
[6, 7, 8]


In [ ]:
data.nunique()

indent_date                                       1
client_id                                         1
store_id                                          1
region_id                                         1
zone_id                                           1
peak_id                                           1
sku_id                                          553
crate_type                                        1
subsection_type                                   4
subsection_id                                   609
order_type                                        2
qty_in_subsection                                40
orientation(along_crate_lbh)                    459
expected_order_composition                       42
raw_sku_order_distribution                       18
raw_expected_sku_order_distribution             188
rationalised_order_qty_raw_order_qty_map         44
rationalised_sku_order_distribution               1
rationalised_expected_sku_order_distribution     41
subsection_p

In [ ]:
import json
x = "{2:[1,2,4]}"
c = eval(x)
print(list(c.keys()))
print(type(c))

[2]
<class 'dict'>


In [ ]:
l = []
for i in range(len(data['peak_id'].unique())):
    l.append(data[data['peak_id']==data['peak_id'].unique()[i]])

In [ ]:
l[0]

,indent_date,client_id,store_id,region_id,zone_id,peak_id,sku_id,crate_type,subsection_type,subsection_id,order_type,qty_in_subsection,orientation(along_crate_lbh),expected_order_composition,raw_sku_order_distribution,raw_expected_sku_order_distribution,rationalised_order_qty_raw_order_qty_map,rationalised_sku_order_distribution,rationalised_expected_sku_order_distribution,subsection_peak_time_order_affinity
0,20221128,zepto,blr_whitefield,1,1,2,88449a9b_7f66_49a6_b24f_9d282ad4a763,LARGE_CRATE,QUARTER_SUBSECTION,1,BULK,284.0,"(143, 78, 36)",{4: 71.07718894009219},{4: 15.571428571428573},"{4: 28.542627728110602, 6: 23.557604686635948,...","{4: [4, 6, 8, 12]}",na,{4: 71.07718894009219},0
1,20221128,zepto,blr_whitefield,1,1,2,ec75d477_2f83_4015_92bd_57dc0f777076,LARGE_CRATE,OCTET_SUBSECTION,3,BULK,88.0,"(120, 40, 5)",{3: 29.333333333333332},{3: 6.666666666666667},"{4: 3.1428581428571434, 2: 23.25714385714286, ...","{3: [4, 2, 3, 6, 1]}",na,{3: 29.333333333333332},0
2,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,5,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
3,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,6,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
4,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,7,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,20221128,zepto,blr_whitefield,1,1,2,50822258_6f8e_49fe_90c7_b2dcdb919bf0,LARGE_CRATE,QUARTER_SUBSECTION,839,BULK,3.0,"(133, 72, 30)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
605,20221128,zepto,blr_whitefield,1,1,2,b5c7c649_9f3f_470c_a32f_7206a9b9f4c0,LARGE_CRATE,QUARTER_SUBSECTION,840,BULK,2.0,"(144, 184, 14)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
606,20221128,zepto,blr_whitefield,1,1,2,de435f90_adbf_41fb_92fc_af03a6bc1dd1,LARGE_CRATE,QUARTER_SUBSECTION,841,BULK,2.0,"(116, 169, 16)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
607,20221128,zepto,blr_whitefield,1,1,2,eef6590f_0ff3_4f2a_9374_099f5871608f,LARGE_CRATE,OCTET_SUBSECTION,842,BULK,2.0,"(145, 96, 13)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0


In [ ]:
m = []
for i in range(len(l)):
    for j in range(len(l[i]['crate_type'].unique())):
        m.append(l[i][l[i]['crate_type']==l[i]['crate_type'].unique()[j]])

In [ ]:
m[0]

,indent_date,client_id,store_id,region_id,zone_id,peak_id,sku_id,crate_type,subsection_type,subsection_id,order_type,qty_in_subsection,orientation(along_crate_lbh),expected_order_composition,raw_sku_order_distribution,raw_expected_sku_order_distribution,rationalised_order_qty_raw_order_qty_map,rationalised_sku_order_distribution,rationalised_expected_sku_order_distribution,subsection_peak_time_order_affinity
0,20221128,zepto,blr_whitefield,1,1,2,88449a9b_7f66_49a6_b24f_9d282ad4a763,LARGE_CRATE,QUARTER_SUBSECTION,1,BULK,284.0,"(143, 78, 36)",{4: 71.07718894009219},{4: 15.571428571428573},"{4: 28.542627728110602, 6: 23.557604686635948,...","{4: [4, 6, 8, 12]}",na,{4: 71.07718894009219},0
1,20221128,zepto,blr_whitefield,1,1,2,ec75d477_2f83_4015_92bd_57dc0f777076,LARGE_CRATE,OCTET_SUBSECTION,3,BULK,88.0,"(120, 40, 5)",{3: 29.333333333333332},{3: 6.666666666666667},"{4: 3.1428581428571434, 2: 23.25714385714286, ...","{3: [4, 2, 3, 6, 1]}",na,{3: 29.333333333333332},0
2,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,5,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
3,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,6,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
4,20221128,zepto,blr_whitefield,1,1,2,208bdb3c_da54_493a_9cb1_5b6c159ae9e1,LARGE_CRATE,HALF_SUBSECTION,7,LIGHT,2.0,"(142, 196, 33)",{1: 71.0},{1: 17.0},"{1: 44.747900159663864, 2: 13.126051420168068}","{1: [1, 2]}",na,{1: 71.0},0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,20221128,zepto,blr_whitefield,1,1,2,50822258_6f8e_49fe_90c7_b2dcdb919bf0,LARGE_CRATE,QUARTER_SUBSECTION,839,BULK,3.0,"(133, 72, 30)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
605,20221128,zepto,blr_whitefield,1,1,2,b5c7c649_9f3f_470c_a32f_7206a9b9f4c0,LARGE_CRATE,QUARTER_SUBSECTION,840,BULK,2.0,"(144, 184, 14)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
606,20221128,zepto,blr_whitefield,1,1,2,de435f90_adbf_41fb_92fc_af03a6bc1dd1,LARGE_CRATE,QUARTER_SUBSECTION,841,BULK,2.0,"(116, 169, 16)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
607,20221128,zepto,blr_whitefield,1,1,2,eef6590f_0ff3_4f2a_9374_099f5871608f,LARGE_CRATE,OCTET_SUBSECTION,842,BULK,2.0,"(145, 96, 13)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0


In [ ]:
n = []
for i in range(len(m)):
    for j in range(len(m[i]['subsection_type'].unique())):
        n.append(m[i][m[i]['subsection_type']==m[i]['subsection_type'].unique()[j]])

In [ ]:
n[0]

,indent_date,client_id,store_id,region_id,zone_id,peak_id,sku_id,crate_type,subsection_type,subsection_id,order_type,qty_in_subsection,orientation(along_crate_lbh),expected_order_composition,raw_sku_order_distribution,raw_expected_sku_order_distribution,rationalised_order_qty_raw_order_qty_map,rationalised_sku_order_distribution,rationalised_expected_sku_order_distribution,subsection_peak_time_order_affinity
0,20221128,zepto,blr_whitefield,1,1,2,88449a9b_7f66_49a6_b24f_9d282ad4a763,LARGE_CRATE,QUARTER_SUBSECTION,1,BULK,284.0,"(143, 78, 36)",{4: 71.07718894009219},{4: 15.571428571428573},"{4: 28.542627728110602, 6: 23.557604686635948,...","{4: [4, 6, 8, 12]}",na,{4: 71.07718894009219},0
66,20221128,zepto,blr_whitefield,1,1,2,32fc13dc_da18_4096_9d38_515d2b60aed3,LARGE_CRATE,QUARTER_SUBSECTION,75,BULK,40.0,"(143, 78, 36)",{3: 13.333333333333334},{3: 3.0},"{1: 31.476191476190476, 2: 4.444445444444445}","{3: [1, 2]}",na,{3: 13.333333333333334},0
71,20221128,zepto,blr_whitefield,1,1,2,a6016a0d_f3de_46d2_a1e8_88a64079db56,LARGE_CRATE,QUARTER_SUBSECTION,85,BULK,31.0,"(121, 68, 30)",{3: 10.333333333333334},{3: 2.0},"{1: 27.047620047619052, 3: 0.7380962380952382,...","{3: [1, 3, 2]}",na,{3: 10.333333333333334},0
72,20221128,zepto,blr_whitefield,1,1,2,f149807f_f7b2_4a6d_9e4b_37d99fc50815,LARGE_CRATE,QUARTER_SUBSECTION,87,BULK,30.0,"(101, 79, 35)",{3: 10.0},{3: 2.3333333333333335},"{1: 3.4489805918367344, 2: 13.357143857142857}","{3: [1, 2]}",na,{3: 10.0},0
75,20221128,zepto,blr_whitefield,1,1,2,02077e52_d4c4_4a30_8885_28406f844488,LARGE_CRATE,QUARTER_SUBSECTION,93,BULK,24.0,"(16, 152, 32)",{3: 8.0},{3: 1.6666666666666667},"{5: 1.1714295714285712, 1: 1.3714295714285711,...","{3: [5, 1, 3, 4, 8]}",na,{3: 8.0},0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,20221128,zepto,blr_whitefield,1,1,2,b8ed990c_6d34_4644_bc40_d9f521acf068,LARGE_CRATE,QUARTER_SUBSECTION,826,BULK,3.0,"(143, 59, 32)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
592,20221128,zepto,blr_whitefield,1,1,2,9b739899_6e4e_4278_b5d5_689a340848db,LARGE_CRATE,QUARTER_SUBSECTION,827,BULK,3.0,"(129, 61, 24)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
604,20221128,zepto,blr_whitefield,1,1,2,50822258_6f8e_49fe_90c7_b2dcdb919bf0,LARGE_CRATE,QUARTER_SUBSECTION,839,BULK,3.0,"(133, 72, 30)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
605,20221128,zepto,blr_whitefield,1,1,2,b5c7c649_9f3f_470c_a32f_7206a9b9f4c0,LARGE_CRATE,QUARTER_SUBSECTION,840,BULK,2.0,"(144, 184, 14)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0


In [ ]:
a = []
for i in range(len(n)):
    for j in range(len(n[i]['order_type'].unique())):
        a.append(n[i][n[i]['order_type']==n[i]['order_type'].unique()[j]])

In [ ]:
a[0]

,indent_date,client_id,store_id,region_id,zone_id,peak_id,sku_id,crate_type,subsection_type,subsection_id,order_type,qty_in_subsection,orientation(along_crate_lbh),expected_order_composition,raw_sku_order_distribution,raw_expected_sku_order_distribution,rationalised_order_qty_raw_order_qty_map,rationalised_sku_order_distribution,rationalised_expected_sku_order_distribution,subsection_peak_time_order_affinity
0,20221128,zepto,blr_whitefield,1,1,2,88449a9b_7f66_49a6_b24f_9d282ad4a763,LARGE_CRATE,QUARTER_SUBSECTION,1,BULK,284.0,"(143, 78, 36)",{4: 71.07718894009219},{4: 15.571428571428573},"{4: 28.542627728110602, 6: 23.557604686635948,...","{4: [4, 6, 8, 12]}",na,{4: 71.07718894009219},0
66,20221128,zepto,blr_whitefield,1,1,2,32fc13dc_da18_4096_9d38_515d2b60aed3,LARGE_CRATE,QUARTER_SUBSECTION,75,BULK,40.0,"(143, 78, 36)",{3: 13.333333333333334},{3: 3.0},"{1: 31.476191476190476, 2: 4.444445444444445}","{3: [1, 2]}",na,{3: 13.333333333333334},0
71,20221128,zepto,blr_whitefield,1,1,2,a6016a0d_f3de_46d2_a1e8_88a64079db56,LARGE_CRATE,QUARTER_SUBSECTION,85,BULK,31.0,"(121, 68, 30)",{3: 10.333333333333334},{3: 2.0},"{1: 27.047620047619052, 3: 0.7380962380952382,...","{3: [1, 3, 2]}",na,{3: 10.333333333333334},0
72,20221128,zepto,blr_whitefield,1,1,2,f149807f_f7b2_4a6d_9e4b_37d99fc50815,LARGE_CRATE,QUARTER_SUBSECTION,87,BULK,30.0,"(101, 79, 35)",{3: 10.0},{3: 2.3333333333333335},"{1: 3.4489805918367344, 2: 13.357143857142857}","{3: [1, 2]}",na,{3: 10.0},0
75,20221128,zepto,blr_whitefield,1,1,2,02077e52_d4c4_4a30_8885_28406f844488,LARGE_CRATE,QUARTER_SUBSECTION,93,BULK,24.0,"(16, 152, 32)",{3: 8.0},{3: 1.6666666666666667},"{5: 1.1714295714285712, 1: 1.3714295714285711,...","{3: [5, 1, 3, 4, 8]}",na,{3: 8.0},0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,20221128,zepto,blr_whitefield,1,1,2,b8ed990c_6d34_4644_bc40_d9f521acf068,LARGE_CRATE,QUARTER_SUBSECTION,826,BULK,3.0,"(143, 59, 32)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
592,20221128,zepto,blr_whitefield,1,1,2,9b739899_6e4e_4278_b5d5_689a340848db,LARGE_CRATE,QUARTER_SUBSECTION,827,BULK,3.0,"(129, 61, 24)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
604,20221128,zepto,blr_whitefield,1,1,2,50822258_6f8e_49fe_90c7_b2dcdb919bf0,LARGE_CRATE,QUARTER_SUBSECTION,839,BULK,3.0,"(133, 72, 30)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0
605,20221128,zepto,blr_whitefield,1,1,2,b5c7c649_9f3f_470c_a32f_7206a9b9f4c0,LARGE_CRATE,QUARTER_SUBSECTION,840,BULK,2.0,"(144, 184, 14)",{3: 0.3333333333333333},{},{},{},na,{3: 0.3333333333333333},0


In [ ]:
order_df

,order_time,store_id,client_id,order_date,order_id,sku_name,sku_id,ordered_units
0,8.0,blr_whitefield,zepto,20221125,1ed9fbwfs99858,pepsi soft drink,82675cec_f35e_4f33_86c9_39d6b0b94f39,1
1,25.0,blr_whitefield,zepto,20221123,e229ebwfs74169,thums up soft drink can,1bb00cdd_62a1_43f4_9ea4_8c55c550896a,2
2,25.0,blr_whitefield,zepto,20221123,e229ebwfs74169,parliament tropical splash,893823a1_131c_40c4_8559_eb6baa94f999,1
3,25.0,blr_whitefield,zepto,20221123,e229ebwfs74169,thums up soft drink,ceeb4c88_2fe4_4c37_9d28_e9d97fd0a726,1
4,46.0,blr_whitefield,zepto,20221125,1ed9fbwfs09962,classic verve cigarette,282941e4_a22a_4380_9d36_6da69319c2ea,3
...,...,...,...,...,...,...,...,...
64651,NaN,blr_whitefield,zepto,20221127,6817fbwfp98088,papas green peas,22557fae_17a9_4c03_ac37_690247e3023d,1
64652,NaN,blr_whitefield,zepto,20221127,6817fbwfp98088,papas green moong dal whole / sabut,64d76860_801f_466f_9d29_bf8f44fadb52,1
64653,NaN,blr_whitefield,zepto,20221127,6817fbwfp98088,popular essentials black masoor whole (un_poli...,8d29340c_a704_45e0_b615_3c8bc8ce0778,1
64654,NaN,blr_whitefield,zepto,20221127,6817fbwfp98088,id idli & dosa batter,a66ba5b4_313d_4469_becd_01b69bd19fc7,1


In [ ]:
#for finding matrix
#diagonals = 0(case 1)
#if ith and jth sku have same sku id then 0(case 2)
#else find no.of times both skus were ordered together from order_history and put that value in matrix at (i,j)(case 3)
#to find no.of times both skus were ordered together in what quantities, use rationalised_order_qty_raw_order_qty_map(dict) and find how many times the key of ith sku was
#ordered with the values of the dict in the jth sku and sum all those values

In [ ]:
sku_id = []
cluster_id = []
cluster_rank = []
group = order_df.groupby(['sku_id','ordered_units'])['order_id'].apply(list).to_dict() #created a group of sku_ids on the basis of order_id
for k in range(len(a)):
  m = len(a[k])
  arr = np.zeros([m,m])
  for i in range(len(a[k])):
    for j in range(len(a[k])):
      #for case 1 and 2
      id_1 = a[k].iloc[i]['sku_id']
      id_2 = a[k].iloc[j]['sku_id']
      sku_id.append(id_1)
      sku_id.append(id_2)
      #u_1 = a[k].iloc[i]['']
      if id_1==id_2:
        arr[i,j]=0
      #case 3
      else:
        qik = list(eval(a[k].iloc[i]['rationalised_order_qty_raw_order_qty_map']).keys())
        qjk = list(eval(a[k].iloc[j]['rationalised_order_qty_raw_order_qty_map']).keys())
        qiv = [element for sublist in list(eval(a[k].iloc[i]['rationalised_order_qty_raw_order_qty_map']).values()) for element in sublist]
        qjv = [element for sublist in list(eval(a[k].iloc[j]['rationalised_order_qty_raw_order_qty_map']).values()) for element in sublist]

        for r in qik:
          for s in qjv:
            o1 = [group.get(id_1,r)]
            o2 = [group.get(id_2,s)]
            for o in o1:
              for g in o2:
                if o==g:
                  arr[i,j]+=1
        for r in qiv:
          for s in qjk:
            o1 = [group.get(id_1,r)]
            o2 = [group.get(id_2,s)]
            for o in o1:
              for g in o2:
                if o==g:
                  arr[i,j]+=1

        for r in qik:
          for s in qjk:
            o1 = [group.get(id_1,r)]
            o2 = [group.get(id_2,s)]
            for o in o1:
              for g in o2:
                if o==g:
                  arr[i,j]-=1

  row_index,column_index = np.where(arr>0)

  edges = []
  nodes = []
  for i in range(len(row_index)):
      index1 = row_index[i]
      index2 = column_index[i]

      sku1 = sku_id[index1]
      sku2 = sku_id[index2]
      edge = (sku1,sku2,arr[index1,index2])
      edges.append(edge)

      if sku1 in nodes:
          pass
      else:
          nodes.append(sku1)
      if sku2 in nodes:
          pass
      else:
          nodes.append(sku2)

  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_weighted_edges_from(edges)

  leiden_com = ig.Graph.from_networkx(G)
  partition = la.find_partition(leiden_com, la.ModularityVertexPartition, n_iterations = -1, weights ='weight')
  cl = len(cluster_id)
  for i, community in enumerate(partition):
    # print(f"Community {i}:")
    for v in community:
        sku_id.append(nodes[v])
        cluster_id.append(i+1+cl)
        cluster_rank.append(i+1)